In [1]:

!pip install -q gradio transformers sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.6 MB/s eta 0:00:00


In [2]:

import gradio as gr
import pandas as pd
import requests
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss


In [3]:

def load_llm():
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

llm = load_llm()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [10]:

csv_path = "/content/simmons_real_mattress_sample.csv"  # Replace with your upload path
df = pd.read_csv(csv_path)

csv_products = []
csv_corpus = []


for _, row in df.iterrows():
    name = row["제품명"]
    price = int(str(row["가격"]).replace(",", "").strip())  # 문자열 가격 처리
    description = row["특징"] if pd.notna(row["특징"]) else ""
    review = row["리뷰요약"] if pd.notna(row["리뷰요약"]) else ""
    csv_products.append({"name": name, "price": price, "link": ""})
    csv_corpus.append(f"{name}: {description} {review}")

In [11]:

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
csv_embeddings = embed_model.encode(csv_corpus)
index = faiss.IndexFlatL2(csv_embeddings.shape[1])
index.add(np.array(csv_embeddings))


In [12]:

def parse_price_range(user_input):
    matches = re.findall(r'\d+', user_input)
    matches = [int(m) for m in matches]
    if not matches:
        return (0, 1000000)
    elif len(matches) == 1:
        val = matches[0] * 10000 if matches[0] < 1000 else matches[0]
        return (0, val)
    elif len(matches) == 2:
        low = matches[0] * 10000 if matches[0] < 1000 else matches[0]
        high = matches[1] * 10000 if matches[1] < 1000 else matches[1]
        return (low, high)
    else:
        return (0, 1000000)

def generate_search_keyword(room_desc, usage_desc):
    keyword = "매트리스"
    if any(x in room_desc for x in ["가족", "아이", "아기", "노부모", "부모님"]):
        keyword = "가족용 매트리스"
    elif "신혼" in room_desc:
        keyword = "신혼 매트리스"
    elif any(x in room_desc for x in ["자취", "원룸"]):
        keyword = "원룸 매트리스"
    elif any(x in room_desc for x in ["청소년", "대학생"]):
        keyword = "1인용 매트리스"
    elif any(x in room_desc for x in ["노인", "노모", "고령"]):
        keyword = "노인용 매트리스"

    if "바닥" in usage_desc:
        keyword = f"바닥용 {keyword}"
    elif "프레임" in usage_desc or "침대" in usage_desc:
        keyword = f"침대용 {keyword}"

    return keyword


In [13]:

NAVER_CLIENT_ID = "T1y5uIaLLnefK1L17oSC"
NAVER_CLIENT_SECRET = "fi9ihoS1xl"

def search_naver_products(keyword, price_range=None, display=5):
    api_url = "https://openapi.naver.com/v1/search/shop.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    params = {"query": keyword, "display": display, "sort": "sim"}
    response = requests.get(api_url, headers=headers, params=params)
    items = response.json().get("items", [])
    results = []
    min_price, max_price = price_range if price_range else (0, float('inf'))
    for item in items:
        price = int(item["lprice"])
        if min_price <= price <= max_price:
            results.append({
                "name": item["title"].replace("<b>", "").replace("</b>", ""),
                "price": price,
                "link": item["link"]
            })
    return results


In [14]:

def init_state():
    return {"room_desc": None, "usage_desc": None, "price_desc": None, "stage": 0}

def chat(user_input, state):
    stage = state["stage"]
    if stage == 0:
        state["room_desc"] = user_input
        state["stage"] += 1
        return "🛏️ 매트리스를 바닥에서 쓸 건가요, 침대 프레임 위에 둘 건가요?", state
    elif stage == 1:
        state["usage_desc"] = user_input
        state["stage"] += 1
        return "💰 예산은 어느 정도로 생각하고 계신가요? (예: 50~100만원)", state
    elif stage == 2:
        state["price_desc"] = user_input
        state["stage"] += 1
        keyword = generate_search_keyword(state["room_desc"], state["usage_desc"])
        price_range = parse_price_range(state["price_desc"])
        naver_products = search_naver_products(keyword, price_range)
        filtered_csv = [p for p in csv_products if price_range[0] <= p["price"] <= price_range[1]]

        query = f"{state['room_desc']} {state['usage_desc']} 매트리스 {price_range}"
        query_emb = embed_model.encode([query])
        D, I = index.search(np.array(query_emb), k=3)
        matched = [csv_corpus[i] for i in I[0]]
        reason = "\n".join([f"• {text}" for text in matched])

        if not naver_products and not filtered_csv:
            return "❌ 조건에 맞는 제품을 찾지 못했어요.", state

        result = f"🛏️ 추천 매트리스 리스트 (예산 {price_range[0]}~{price_range[1]}원 기준):\n\n"
        for i, p in enumerate(naver_products[:3], 1):
            result += f"[네이버] {i}. {p['name']} - {p['price']}원\n🔗 {p['link']}\n"
        for i, p in enumerate(filtered_csv[:3], 1):
            result += f"[샘플] {i}. {p['name']} - {p['price']}원\n🔗 (링크 없음)\n"

        result += f"\n📌 추천 이유 (설명 및 후기 기반 요약):\n{reason}"
        result += "\n🎉 새로운 질문은 '처음부터'라고 입력해주세요."
        return result, state
    else:
        if "처음" in user_input or "다시" in user_input:
            return "🔁 대화를 새로 시작합니다. 사용하실 공간 또는 상황을 말씀해주세요.", init_state()
        return "🙋 추천을 완료했어요. 새로운 질문을 하시려면 '처음부터'라고 입력해주세요.", state


In [15]:

with gr.Blocks() as demo:
    state = gr.State(init_state())
    chatbox = gr.Textbox(label="질문 입력", placeholder="예: 아이가 태어나서 온 가족이 쓸 매트리스를 추천해줘")
    output = gr.Textbox(label="AI 응답", interactive=False)
    chatbox.submit(chat, inputs=[chatbox, state], outputs=[output, state])
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5dc0b9a557edd13a8e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
print(df.columns)

Index(['제품명', '브랜드', '가격', '사이즈', '경도', '특징', '리뷰요약'], dtype='object')
